In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
import tqdm
from matplotlib.pyplot import figure
import torch
import torchvision as tv
%matplotlib inline

## Implementation of own convolution

In [3]:
def my_conv2d(x,
              kernel,
              stride=(1,1), padding=(0,0), bias=None):
    """
    Arguments:
    :param  x: input tensor 4d, type tensor.FloatTensor, BxCxHxW,
    :param  kernel: input kernel tensor 3d, type tensor.FloatTensor, CxHxW,
    :param  stride: tuple - stride parameters, set in HxW format,
    :param  padding:  tuple - padding parameters, set in HxW format,
    :param  bias : the input tensor bias is added to the output tensor,
    
    where:
    B is the batch or the number of input,
    C is the number of image channels,
    H is the height of image,
    W is is the width of the image.
    """
    
    # getting input parameters
    print(x.size())
    in_s = x.size()
    if x.dim() == 3:
        b_in = 1
        c_in = in_s[0]
        h_in = in_s[1]
        w_in = in_s[2]
    elif x.dim() == 4:
        b_in = in_s[0]
        c_in = in_s[1]
        h_in = in_s[2]
        w_in = in_s[3]
    else:
        raise Exception("ERROR: Wrong input tensor. Input tensor must be 3d or 4d.")
        
    k_s = kernel.size()
    k_c = k_s[0]
    k_h = k_s[1]
    k_w = k_s[2]
    
    # output size calculation
    h_out = (h_in + 2 * padding[0] - k_h) / stride[0] + 1
    w_out = (w_in + 2 * padding[1] - k_w) / stride[1] + 1
    
    if not h_out.is_integer() or not w_out.is_integer():
        print("Wrong output dimension. H_out = {}, W_out = {}. Some input units will be left out.".format(h_out, w_out))
        h_out = (h_in + 2 * padding[0] - k_h) // stride[0] + 1
        w_out = (w_in + 2 * padding[1] - k_w) // stride[1] + 1
        print("New output sizes: H_out = {}, W_out = {}".format(h_out, w_out))
    #else:
    h_out = int(h_out)
    w_out = int(w_out)
        
    # adding padding
    x = torch.from_numpy(np.pad(x.numpy(),
                                ((0, 0), (padding[0],padding[0]), (padding[1],padding[1])),
                                mode='constant'))
    
    # output tensor
    x_out = torch.zeros(h_out, w_out)
    print(x_out.size())
    
    # iterating over input tensor
    #for b in range(b_in):
    for c in range(c_in):
        for h in range(0, h_in + 2 * padding[0] - k_h + 1, stride[0]):
            #print("h = ", h)
            for w in range(0, w_in + 2 * padding[1] - k_w + 1, stride[1]):
                res = 0
                for kc in range(k_c):
                    for kh in range(k_h):
                        for kw in range(k_w):
                            res += x[c][h + kh][w + kw] * kernel[kc][kh][kw]
                x_out[int(h/stride[0])][int(w/stride[1])] += (res / k_c)
                            
    return x_out

In [9]:
import torch

t = torch.tensor([[
    [1,2,3,4],
    [5,6,7,8],
    [9,10,11,12],
    [13,14,15,16]
],[
    [1,2,3,4],
    [5,6,7,8],
    [9,10,11,12],
    [13,14,15,16]
],[
    [1,2,3,4],
    [5,6,7,8],
    [9,10,11,12],
    [13,14,15,16]
]
], dtype=torch.float32)



print(t)
print(t[0][0][0])
#print(t.size())
#print(t.numel())

padding = (2, 3)
#x_padded = np.pad(t[0], ((0, 0), (0, 0), (p, p), (p, p)), mode='constant')
t = np.pad(t, ((0, 0), (padding[0],padding[0]), (padding[1],padding[1])), mode='constant')

print(t)
print(t[0][2][3])

#filter_kernel = torch.FloatTensor(3,1,1).fill_(-1)
#filter_kernel[:,1,1] = 8

#print(filter_kernel)

#my_conv2d(t, filter_kernel)

tensor([[[ 1.,  2.,  3.,  4.],
         [ 5.,  6.,  7.,  8.],
         [ 9., 10., 11., 12.],
         [13., 14., 15., 16.]],

        [[ 1.,  2.,  3.,  4.],
         [ 5.,  6.,  7.,  8.],
         [ 9., 10., 11., 12.],
         [13., 14., 15., 16.]],

        [[ 1.,  2.,  3.,  4.],
         [ 5.,  6.,  7.,  8.],
         [ 9., 10., 11., 12.],
         [13., 14., 15., 16.]]])
tensor(1.)
[[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  1.  2.  3.  4.  0.  0.  0.]
  [ 0.  0.  0.  5.  6.  7.  8.  0.  0.  0.]
  [ 0.  0.  0.  9. 10. 11. 12.  0.  0.  0.]
  [ 0.  0.  0. 13. 14. 15. 16.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  1.  2.  3.  4.  0.  0.  0.]
  [ 0.  0.  0.  5.  6.  7.  8.  0.  0.  0.]
  [ 0.  0.  0.  9. 10. 11. 12.  0.  0.  0.]
  [ 0.  0.  0. 13. 14. 15. 16.  0.  0